In [1]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType, TimestampType
from pyspark.sql.functions import unix_timestamp
import pandas as pd

# import numpy as np
# import matplotlib.pyplot as mplt
# import seaborn as sns
# from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram

In [2]:
divvystation = sqlContext.read.format("csv").options(header='true', inferSchema='true').load("/FileStore/tables/Divvy_Stations_2016_Q4.csv")
funcd =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
divvystation = divvystation.withColumn('online_date', funcd(col('online_date')))
divvystation.show(3)
# display(divvystation)

In [3]:
divvytrips = sqlContext.read.format("csv").options(header='true', inferSchema='true').load("/FileStore/tables/divvyride/Divvy_Trips_2016_Q4.csv")
# Date formatting at http://strftime.org/
funcdt =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), TimestampType())
divvytrips = divvytrips.withColumn('starttime', funcdt(col('starttime')))
divvytrips = divvytrips.withColumn('stoptime', funcdt(col('stoptime')))
divvytrips.show(3)

In [4]:
divvytrips.printSchema()

In [5]:
divvystation.printSchema()

In [6]:
divvytrips2 = divvytrips.select("trip_id","bikeid","tripduration","from_station_id","to_station_id","usertype","gender","birthyear")
divvytrips2.agg({'tripduration':'mean'}).show()
divvytrips2.agg({'tripduration':'max'}).show()

In [7]:
# Trip Duration By User Type
display(divvytrips2)

In [8]:
divvytrips3 = divvytrips2.groupby('from_station_id').agg({'tripduration':'count'})
divvytrips3 = divvytrips3.withColumnRenamed("count(tripduration)","counttd")
divvytrips3.orderBy(divvytrips3.counttd.desc()).limit(20).join(divvystation, divvytrips3.from_station_id == divvystation.id).show()


In [9]:
# Count of Rides from Top Stations
display(divvytrips3.orderBy(divvytrips3.counttd.desc()).limit(25))

In [10]:
divvymerge = divvytrips3.join(divvystation, divvytrips3.from_station_id == divvystation.id)
# <Data.Frame>.toPandas

In [11]:
display(divvymerge.select('dpcapacity','counttd'))

In [12]:
# Avg Trip Time By BikeID
avgtrips = divvytrips2.groupby('bikeid').agg({'tripduration':'mean'})
avgtrips = avgtrips.withColumnRenamed("avg(tripduration)","avgtrip_in_s")
avgtrips.orderBy(avgtrips.bikeid.asc()).show(10)

In [13]:
# Distribution of Avg Trip Time By BikeID
display(avgtrips)